## Agentic RAG + LangGraph

에이전트(Agent)는 검색 도구를 사용할지 여부를 결정해야할때 유용하다. 에이전트와 관련된 내용은 [Agent](https://wikidocs.net/233782)페이지를 참고.

검색 에이전트를 구현하기 위해서 `LLM`에 검색 도구에 대한 접근 권한을 부여하기만 하면된다.

그리고 이를 [LangGraph](https://langchain-ai.github.io/langgraph/)에 통합할 수있다.

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

### 기본 PDF 기반 Retrieval Chain 생성

PDF 문서를 기반으로 Retrieval Chain을 생성한다. 가장 단순한 구조의 Retrieval Chain.

단, LangGraph 에서는 Retriever과 Chain 을 따로 생성한다. 그래야 각 노드별로 세부 처리를 할 수 있다.

In [17]:
from rag.pdf import PDFRetrievalChain

# pdf 문서 로드
pdf = PDFRetrievalChain(["docs/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

# retriever와 chain을 생성
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

AttributeError: 'PDFRetrievalChain' object has no attribute 'load_documents'